In [1]:

def create_equity_curve(df,z0,z1,pos):
    ec=df.diff(axis=0)
    ec.iloc[0]=0
    
    i=1
    while i<len(ec):
        ec.iloc[i]=ec.iloc[i-1]+ec.iloc[i]*z0*z1*pos
        i+=1
    return ec
    
    

def pos():
    pos=1
    while True:
        a=raw_input("Please input 'y' if position is long and 'n' if position is short: ").lower()
    
        if a=='y':
            break
        if a=='n':
            pos=pos*(-1)
            break
        else:
            print "Please input either 'y' or 'n' for position"
    return pos

def z_factor(product1,product2):
    z=1
    factor=1000
    if product1=='RB' and product2 =='RB':
        z=42
    elif product1=='HO' and product2 =='HO':
        z=42
    elif product1=='RB' and product2 =='HO':
        z=42
    elif product1=='HO' and product2 =='RB':
        z=42
    elif product1=='G' and product2 =='G':
        factor=100
    elif product1=='G' and product2 =='B':
        z=(1/7.45)
    
    else:
        z=1
        factor=1000
    
    return z,factor
        
def create_sprd(dataframe1, dataframe2):
   
    dataframe1.reset_index(inplace=True)
    dataframe2.reset_index(inplace=True)
    date=dataframe1['Date']
    
    dataframe1.drop(columns=['Date'],inplace=True)
    dataframe2.drop(columns=['Date'],inplace=True)
    
    while True:
        sprd_type=int(raw_input('Please indicate if this is a 1) Crack or 2) Spread - please input integer 1 or 2: '))
        
        if sprd_type==1:
            
            new_df=dataframe1*42-dataframe2
            new_df['Date']=date
            new_df.set_index('Date',inplace=True)
            new_df.fillna(method='bfill',inplace=True)
            new_df.fillna(method='ffill',inplace=True)
            
            return new_df
            break
        
        elif sprd_type==2:
            
            new_df= dataframe1-dataframe2
            new_df['Date']=date
            new_df.set_index('Date',inplace=True)
            new_df.fillna(method='bfill',inplace=True)
            new_df.fillna(method='ffill',inplace=True)
            
            return new_df
            break
        
        else:
            print 'You have inputted incorrectly. Please try again and enter either integer 1 or 2.'
    

class Mth_1(object):
    
    def __init__(self):
        pass
        
    def pdt_type1(self):
        
        product=''
        while True:
            inst=int(raw_input('Please input the number of the product you want to see the seasonals: 1) Brent 2) Gasoil 3) HO 4) RBOB 5) WTI: '))
    
            if inst==1:
                product='B'
                break
            elif inst==2:
                product='G'
                break   
            elif inst==3:
                product='HO'
                break      
            elif inst==4:
                product='RB'
                break       
            elif inst==5:
                product='CL'
                break       
            else:
                print 'You did not input a correct number. Please try again.'
        
        return product
        
    
    def mth_no1(self):
        month=''
        while True:
            Mth=int(raw_input('Please input the month of the contract which you wish to evaluate in numbers from 1-12: '))
    
            if Mth==1:
                month='Jan'
                break
            elif Mth==2:
                month='Feb'
                break   
            elif Mth==3:
                month='Mar'
                break   
            elif Mth==4:
                month='Apr'
                break   
            elif Mth==5:
                month='May'
                break          
            elif Mth==6:
                month='Jun'
                break   
            elif Mth==7:
                month='Jul'
                break   
            elif Mth==8:
                month='Aug'
                break   
            elif Mth==9:
                month='Sep'
                break   
            elif Mth==10:
                month='Oct'
                break   
            elif Mth==11:
                month='Nov'
                break   
            elif Mth==12:
                month='Dec'
                break 
            else:
                print 'You did not input a correct number. Please try again.'
        
        return month

    def dataframe_create(self,start_mth,start_day,end_mth,end_day,i):

        count=0
        path = os.getcwd()
        
        listing = os.listdir(path)
        for infile in listing:
            temp=pd.read_excel(infile) #read each file into panda dataframe
    
            contract_yr_str=infile[-8:-4] # get the contract year
    
            seasonal_year=int(contract_yr_str)-1 # get the seasonal year to create the index - prev year
            seasonal_year=str(seasonal_year) # convert to string

            start_date=str(int(seasonal_year)+i)+'-'+start_mth+'-'+start_day # create start date
            end_date=str(int(seasonal_year)+j)+'-'+end_mth+'-'+end_day # create end date
            
            #Join a date dataframe and the temp dataframe together
            
            dates=pd.date_range(start_date, end_date) # range of dates for date axis
            dates=dates.strftime('%Y-%m-%d') #make sure no left out dates
            sample_data=np.arange(0,len(dates)) #just sample numbers to create dataframe
    
            dates_dataframe=pd.DataFrame({'Date':dates,'data':sample_data}) # create dates dataframe
            dates_dataframe=dates_dataframe.set_index('Date') # set index as Date to facilitate joining
    
            temp=temp.set_index('Date') # set index as Date to facilitate joining
    
            temp=dates_dataframe.join(temp) # join dataframes together on Date
               
            #filter dates by the start and end date
            temp.reset_index(inplace=True)
    
            temp=temp[temp['Date']>=start_date] # filter for start date

            temp=temp[temp['Date']<=end_date] # filter for end date

            temp['Date']=pd.to_datetime(temp['Date']).dt.strftime('%m/%d') # putting date in mm/dd form
            temp=temp.set_index('Date') # putting date back as index
            temp=pd.DataFrame(temp['Settle'],columns=['Settle']) # setting dataframe with date and settle only

    
            if count==0:
                Seasonal_Combi=temp
                seasonal_year=str(int(seasonal_year)+1)
                Seasonal_Combi.rename(columns={'Settle':seasonal_year},inplace=True)
        
                count+=1
        
            else:
    
                seasonal_year=str(int(seasonal_year)+1)
                Seasonal_Combi['Settle']=temp['Settle']
                Seasonal_Combi.rename(columns={'Settle':seasonal_year},inplace=True)

        #don't need this, i believe
        #Seasonal_Combi.drop(columns=['Open','High','Low','Last','Change', 'Volume','Prev. Day Open Interest'],inplace=True)   

        
        return Seasonal_Combi
            
            

class Mth_2(object):
    
    def __init__(self):
        pass
        
    def pdt_type2(self):
        
        product=''
        while True:
            inst=int(raw_input('Please input the number of the product you want to see the seasonals: 1) Brent 2) Gasoil 3) HO 4) RBOB 5) WTI: '))
    
            if inst==1:
                product='B'
                break
            elif inst==2:
                product='G'
                break   
            elif inst==3:
                product='HO'
                break      
            elif inst==4:
                product='RB'
                break       
            elif inst==5:
                product='CL'
                break       
            else:
                print 'You did not input a correct number. Please try again.'
        
        return product
        
    
    def mth_no2(self):
        month=''
        while True:
            Mth=int(raw_input('Please input the month of the contract which you wish to evaluate in numbers from 1-12: '))
    
            if Mth==1:
                month='Jan'
                break
            elif Mth==2:
                month='Feb'
                break   
            elif Mth==3:
                month='Mar'
                break   
            elif Mth==4:
                month='Apr'
                break   
            elif Mth==5:
                month='May'
                break          
            elif Mth==6:
                month='Jun'
                break   
            elif Mth==7:
                month='Jul'
                break   
            elif Mth==8:
                month='Aug'
                break   
            elif Mth==9:
                month='Sep'
                break   
            elif Mth==10:
                month='Oct'
                break   
            elif Mth==11:
                month='Nov'
                break   
            elif Mth==12:
                month='Dec'
                break 
            else:
                print 'You did not input a correct number. Please try again.'
        
        return month

    def dataframe_create(self, start_mth, start_day, end_mth,end_day,i):

        count=0
        path = os.getcwd()

        listing = os.listdir(path)
        for infile in listing:
            temp=pd.read_excel(infile) #read each file into panda dataframe
    
            contract_yr_str=infile[-8:-4] # get the contract year
    
            seasonal_year=int(contract_yr_str)-1 # get the seasonal year to create the index - prev year
            seasonal_year=str(seasonal_year) # convert to string
     
            start_date=str(int(seasonal_year)+i)+'-'+start_mth+'-'+start_day # create start date
            end_date=str(int(seasonal_year)+j)+'-'+end_mth+'-'+end_day # create end date
            
            #Join a date dataframe and the temp dataframe together
            
            dates=pd.date_range(start_date, end_date) # range of dates for date axis
            dates=dates.strftime('%Y-%m-%d') #make sure no left out dates
            sample_data=np.arange(0,len(dates)) #just sample numbers to create dataframe
    
            dates_dataframe=pd.DataFrame({'Date':dates,'data':sample_data}) # create dates dataframe
            dates_dataframe=dates_dataframe.set_index('Date') # set index as Date to facilitate joining
    
            temp=temp.set_index('Date') # set index as Date to facilitate joining
    
            temp=dates_dataframe.join(temp) # join dataframes together on Date
               
            #filter dates by the start and end date
            temp.reset_index(inplace=True)
    
            temp=temp[temp['Date']>=start_date] # filter for start date

            temp=temp[temp['Date']<=end_date] # filter for end date

            temp['Date']=pd.to_datetime(temp['Date']).dt.strftime('%m/%d') # putting date in mm/dd form
            temp=temp.set_index('Date') # putting date back as index
            temp=pd.DataFrame(temp['Settle'],columns=['Settle']) # setting dataframe with date and settle only
    
            if count==0:
                Seasonal_Combi=temp
                seasonal_year=str(int(seasonal_year)+1)
                Seasonal_Combi.rename(columns={'Settle':seasonal_year},inplace=True)
        
                count+=1
        
            else:
    
                seasonal_year=str(int(seasonal_year)+1)
                Seasonal_Combi['Settle']=temp['Settle']
                Seasonal_Combi.rename(columns={'Settle':seasonal_year},inplace=True)

        #Seasonal_Combi.drop(columns=['Open','High','Low','Last','Change','Volume','Prev. Day Open Interest'],inplace=True)   

        
        return Seasonal_Combi
    
    
import numpy as np
import pandas as pd
import os

#from plotly import __version__
#import cufflinks as cf
#from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
#init_notebook_mode(connected=True)
#cf.go_offline()

%matplotlib inline

#user specify instrument and contract month

product1=''
month1=''
product2=''
month2=''

front_mth=Mth_1()
product1 = front_mth.pdt_type1()
month1=front_mth.mth_no1()

back_mth=Mth_2()
product2=back_mth.pdt_type2()
month2=back_mth.mth_no2()

        

Please input the number of the product you want to see the seasonals: 1) Brent 2) Gasoil 3) HO 4) RBOB 5) WTI: 4
Please input the month of the contract which you wish to evaluate in numbers from 1-12: 3
Please input the number of the product you want to see the seasonals: 1) Brent 2) Gasoil 3) HO 4) RBOB 5) WTI: 5
Please input the month of the contract which you wish to evaluate in numbers from 1-12: 3


In [2]:
cd Quandl

C:\Users\boonsin.teo\Documents\Quandl


In [3]:
cd $product1

C:\Users\boonsin.teo\Documents\Quandl\RB


In [4]:
cd $month1

C:\Users\boonsin.teo\Documents\Quandl\RB\Mar


In [5]:
start_mth=raw_input('Please input start mth of seasonal period in MM format: ')
start_day=raw_input('Please input start day of seasonal period in DD format: ')
i=int(raw_input('Please input 1 if start mth is the same as contract year and 0 if not: '))
            
end_mth=raw_input('Please input end mth of seasonal period in MM format: ')
end_day=raw_input('Please input end day of seasonal period in DD format: ')
            
j=int(raw_input('Please input 1 if end mth is the same as contract year and 0 if not: '))


front_month=front_mth.dataframe_create(start_mth,start_day,end_mth,end_day,i)

Please input start mth of seasonal period in MM format: 01
Please input start day of seasonal period in DD format: 01
Please input 1 if start mth is the same as contract year and 0 if not: 1
Please input end mth of seasonal period in MM format: 01
Please input end day of seasonal period in DD format: 15
Please input 1 if end mth is the same as contract year and 0 if not: 1


In [6]:
cd ..

C:\Users\boonsin.teo\Documents\Quandl\RB


In [7]:
cd ..

C:\Users\boonsin.teo\Documents\Quandl


In [8]:
cd ..

C:\Users\boonsin.teo\Documents


In [9]:
cd Quandl

C:\Users\boonsin.teo\Documents\Quandl


In [10]:
cd $product2

C:\Users\boonsin.teo\Documents\Quandl\CL


In [11]:
cd $month2

C:\Users\boonsin.teo\Documents\Quandl\CL\Mar


In [12]:
back_month=back_mth.dataframe_create(start_mth, start_day,end_mth,end_day,i)

In [13]:
cd ..

C:\Users\boonsin.teo\Documents\Quandl\CL


In [14]:
cd ..

C:\Users\boonsin.teo\Documents\Quandl


In [15]:
cd ..

C:\Users\boonsin.teo\Documents


In [16]:
front_month.head()

,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
Date,,,,,,,,,,,,,,,,,,,,,
01/01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
01/02,0.5711,0.7034,0.5401,NaN,NaN,0.7936,0.6191,0.8765,NaN,NaN,...,1.1540,NaN,NaN,NaN,2.7900,2.7087,1.4684,NaN,NaN,1.7848
01/03,0.5695,0.7001,NaN,NaN,NaN,0.8081,0.6117,0.9100,NaN,1.1448,...,NaN,NaN,2.4333,2.7534,2.7936,2.6633,NaN,NaN,1.6405,1.8185
01/04,0.5734,NaN,NaN,0.3855,0.6850,0.8171,0.6367,NaN,NaN,1.1874,...,NaN,2.1201,2.4203,2.7949,2.7682,NaN,NaN,1.3171,1.6631,NaN
01/05,0.5822,NaN,0.5259,0.3753,0.6778,0.8180,NaN,NaN,0.9777,1.1845,...,1.2259,NaN,2.4517,2.7480,NaN,NaN,1.4153,1.2867,1.6581,NaN


In [17]:
back_month.head()

,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
Date,,,,,,,,,,,,,,,,,,,,,
01/01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
01/02,19.24,25.07,17.66,NaN,NaN,26.52,21.23,31.10,NaN,NaN,...,50.21,NaN,NaN,NaN,93.55,95.62,53.11,NaN,NaN,60.38
01/03,19.34,24.99,NaN,NaN,NaN,27.27,20.64,32.22,NaN,42.34,...,NaN,NaN,92.43,103.14,93.34,94.14,NaN,NaN,53.29,61.55
01/04,19.37,NaN,NaN,12.47,24.84,27.54,21.91,NaN,NaN,44.08,...,NaN,82.12,90.42,103.40,93.51,NaN,NaN,37.95,54.22,NaN
01/05,19.65,NaN,17.14,12.14,24.27,27.16,NaN,NaN,33.51,43.61,...,52.69,NaN,91.42,102.00,NaN,NaN,50.52,37.18,54.68,NaN


In [18]:
Spread_Combi=create_sprd(front_month, back_month)

Please indicate if this is a 1) Crack or 2) Spread - please input integer 1 or 2: 1


In [19]:
Spread_Combi.dropna(axis=0,how='all',inplace=True) # drop rows with all NAs

In [20]:
Spread_Combi.dropna(axis=1,how='all',inplace=True)# drop columns with all NAs

In [21]:
position=pos()
print 'PnL will be calculated basis 1kb lot size for standardization purposes.'

Please input 'y' if position is long and 'n' if position is short: y
PnL will be calculated basis 1kb lot size for standardization purposes.


In [22]:
zf=z_factor(product1, product2)

In [23]:
#PnL Series
pnl_series=(Spread_Combi.iloc[-1]-Spread_Combi.iloc[0])*zf[0]*zf[1]*position

In [24]:
#Win %
Win_Percent=float(pnl_series[pnl_series>0].count())/float(pnl_series.count())*100

In [25]:
#Largest Drawdown - use max and min
max_series=Spread_Combi.max()
min_series=Spread_Combi.min()
if position==1:
    max_drawdown=(min_series-Spread_Combi.iloc[0])*position*zf[0]*zf[1]
else:
    max_drawdown=(max_series-Spread_Combi.iloc[0])*position*zf[0]*zf[1]

In [26]:
#% return - calculate using last value vs first value - similar to pnl calc
percent_return=((Spread_Combi.iloc[-1]-Spread_Combi.iloc[0])/(Spread_Combi.iloc[0]))

In [27]:
pnl_series

1996    -460.6
1997    -669.0
1998    -173.0
1999    -574.2
2000     459.4
2001     910.0
2002    -419.2
2003     -27.0
2004    -632.8
2005      34.4
2006   -1106.6
2007      -4.0
2008   -3139.4
2009    8942.2
2010    1183.8
2011    2930.2
2012    3684.6
2013   -2853.8
2014   -1531.4
2015     731.0
2016   -3834.6
2017       5.6
2018     245.4
dtype: float64

In [28]:
percent_return

1996   -0.097046
1997   -0.149571
1998   -0.034433
1999   -0.154313
2000    0.116896
2001    0.133603
2002   -0.087842
2003   -0.004726
2004   -0.083777
2005    0.005991
2006   -0.101678
2007   -0.000535
2008   -0.313539
2009   -5.133295
2010    0.170966
2011    0.299961
2012    0.294702
2013   -0.120770
2014   -0.084396
2015    0.085369
2016   -0.220783
2017    0.000359
2018    0.016829
dtype: float64

In [29]:
max_drawdown

1996    -651.0
1997    -669.0
1998    -173.0
1999    -574.2
2000    -102.0
2001    -141.0
2002    -474.2
2003    -760.4
2004    -632.8
2005    -322.8
2006   -1323.2
2007    -995.0
2008   -3139.4
2009       0.0
2010    -294.4
2011       0.0
2012       0.0
2013   -2853.8
2014   -1531.4
2015    -338.4
2016   -3834.6
2017   -1398.6
2018       0.0
dtype: float64

In [30]:
print 'The winning percentage is %.2f percent' %Win_Percent

the win percentage is 43.48 percent


In [31]:
#Equity_Curve=create_equity_curve(Spread_Combi, zf[0],zf[1],position)

In [32]:
#Equity_Curve.iplot(kind='line')